# Investigation of the impact of causality constraints on a CNN-LSTM sleep staging model

### Project goals:
##### Classify sleep stages using multi-modal sensor data (BVP, accelerometer, timestamps, temperature).
##### Compare model performance and computation between non-causal versus causal architectures.

## Library Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchmetrics.classification import MulticlassCohenKappa
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from collections import Counter
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers.wandb import WandbLogger
import wandb

In [2]:
# demo csv, get columns
demo_csv_path = '/gpfs/data/oermannlab/users/slj9342/dl4med_25/data/physionet.org/files/dreamt/2.0.0/data_64Hz/S016_whole_df.csv'
demo_df = pd.read_csv(demo_csv_path)
print(demo_df.columns)
col_dtypes = demo_df.dtypes
print(col_dtypes)

/tmp/ipykernel_2298590/190206057.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  demo_df = pd.read_csv(demo_csv_path)


Index(['TIMESTAMP', 'BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP', 'EDA', 'HR',
       'IBI', 'Sleep_Stage', 'Obstructive_Apnea', 'Central_Apnea', 'Hypopnea',
       'Multiple_Events'],
      dtype='object')
TIMESTAMP            float64
BVP                  float64
ACC_X                float64
ACC_Y                 object
ACC_Z                float64
TEMP                 float64
EDA                  float64
HR                   float64
IBI                  float64
Sleep_Stage           object
Obstructive_Apnea    float64
Central_Apnea        float64
Hypopnea             float64
Multiple_Events      float64
dtype: object


## Data loading

In [10]:
datadir_64Hz = '/gpfs/data/oermannlab/users/slj9342/dl4med_25/data/physionet.org/files/dreamt/2.0.0/data_64Hz/' # working with 64Hz data

dtype_dict = {
    'TIMESTAMP': np.float32,
    'BVP': np.float32,
    'ACC_X': np.float32,
    'ACC_Y': np.float32,
    'ACC_Z': np.float32,
    'TEMP': np.float32,
    'EDA': np.float32,
    'HR': np.float32,
    'IBI': np.float32,  # <-- missing but important
    'Sleep_Stage': 'category',
    'Obstructive_Apnea': 'Int64',  # Nullable integer
    'Central_Apnea': 'Int64',
    'Hypopnea': 'Int64',
    'Multiple_Events': 'Int64'
}

df_head = pd.read_csv(os.path.join(datadir_64Hz, file), nrows=5)
print(df_head.columns.tolist())

df = pd.read_csv(os.path.join(datadir_64Hz, file))
print(df.dtypes)

df = pd.read_csv(os.path.join(datadir_64Hz, file), low_memory=False)
for col in df.columns:
    try:
        pd.to_numeric(df[col], errors='raise')
    except Exception as e:
        print(f"❌ Column {col} failed: {e}")



['TIMESTAMP', 'BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP', 'EDA', 'HR', 'IBI', 'Sleep_Stage', 'Obstructive_Apnea', 'Central_Apnea', 'Hypopnea', 'Multiple_Events']


/tmp/ipykernel_2298590/2468003919.py:23: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(datadir_64Hz, file))


TIMESTAMP            float64
BVP                  float64
ACC_X                 object
ACC_Y                float64
ACC_Z                float64
TEMP                 float64
EDA                  float64
HR                   float64
IBI                  float64
Sleep_Stage           object
Obstructive_Apnea    float64
Central_Apnea        float64
Hypopnea             float64
Multiple_Events      float64
dtype: object
❌ Column ACC_X failed: Unable to parse string "R" at position 1979585
❌ Column Sleep_Stage failed: Unable to parse string "P" at position 0


In [ ]:
# get max sequence length
def safe_float(x):
    try:
        return float(x)
    except ValueError:
        return np.nan


numeric_columns = [
    'TIMESTAMP', 'BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP',
    'EDA', 'HR', 'IBI'
]
converters = {col: safe_float for col in numeric_columns}
max_length = 0
for file in os.listdir(datadir_64Hz):
    if file.endswith('_whole_df.csv'):
        df = pd.read_csv(
            os.path.join(datadir_64Hz, file),
            dtype={'Sleep_Stage': 'category'},
            converters=converters,
            low_memory=True
        )
        max_length = max(max_length, len(df))
print(f"Max sequence length: {max_length}")

Max sequence length: 2493810


### Split subjects into train, val, and test

In [13]:
participant_info_df = pd.read_csv('/gpfs/data/oermannlab/users/slj9342/dl4med_25/data/physionet.org/files/dreamt/2.0.0/participant_info.csv')
subjects_all = participant_info_df['SID']

subjects_all_shuffled = participant_info_df['SID'].sample(frac=1, random_state=42).reset_index(drop=True)
subjects_train = subjects_all_shuffled[:int(len(subjects_all_shuffled)*0.8)]
subjects_val = subjects_all_shuffled[int(len(subjects_all_shuffled)*0.8):int(len(subjects_all_shuffled)*0.9)]
subjects_test = subjects_all_shuffled[int(len(subjects_all_shuffled)*0.9):]
print(f"number of subjects in train: {len(subjects_train)}")
print(f"number of subjects in val: {len(subjects_val)}")
print(f"number of subjects in test: {len(subjects_test)}")

number of subjects in train: 80
number of subjects in val: 10
number of subjects in test: 10


In [15]:
sample, label, sid1 = demo_dataset[3]
print(f"Sample shape: {sample.shape} (channels, epoch_samples), Label: {label}, Subject ID: {sid1}")

sample, label, sid2 = demo_dataset[3071]
print(f"Sample shape: {sample.shape} (channels, epoch_samples), Label: {label}, Subject ID: {sid2}")

sample, label, sid3 = demo_dataset[12000]
print(f"Sample shape: {sample.shape} (channels, epoch_samples), Label: {label}, Subject ID: {sid3}")

Sample shape: torch.Size([32, 4]) (channels, epoch_samples), Label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), Subject ID: S002
Sample shape: torch.Size([32, 4]) (channels, epoch_samples), Label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), Subject ID: S002
Sample shape: torch.Size([32, 4]) (channels, epoch_samples), Label: tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2]), Subject ID: S002


### Non-windowed dataset class




In [ ]:
SLEEP_STAGE_MAPPING = {
    "W": 0,    # Wake
    "N1": 1,   # non-REM stage 1
    "N2": 2,   # non-REM stage 2
    "N3": 3,   # non-REM stage 3
    "R": 4,    # REM
    "Missing": -1  # Missing label
}

def forward_fill(x):
    """
    Performs forward fill on a tensor. If x is 1D (shape [T]),
    it's temporarily unsqueezed to [T, 1] and then processed.
    Assumes the first value is valid, or fills it with zero if needed.
    """
    single_channel = False
    if x.dim() == 1:
        x = x.unsqueeze(1)
        single_channel = True
    
    T, C = x.shape
    for c in range(C):
        # Optionally, handle the first element if it's NaN
        if torch.isnan(x[0, c]):
            x[0, c] = 0.0  # or choose another default value
        for t in range(1, T):
            if torch.isnan(x[t, c]):
                x[t, c] = x[t - 1, c]
    
    if single_channel:
        x = x.squeeze(1)
    return x

class SleepDataset(Dataset):
    def __init__(self, subjects_list, data_dir, max_length, downsample_freq=64, debug=False):
        self.subjects = [{} for _ in range(len(subjects_list))]
        self.downsample = int(64 // downsample_freq)  # Downsample factor
        self.max_length = int(max_length // self.downsample)

        for subjectNo, SID in enumerate(subjects_list):
            # Load the data for each subject
            file_path = os.path.join(data_dir, f"{SID}_whole_df.csv")
            if os.path.exists(file_path):
                df = pd.read_csv(
                    file_path,
                    dtype={'Sleep_Stage': 'category'},
                    converters=converters,
                    low_memory=True
                )
                if debug:
                    print(f"loaded data for {SID}:")

                # Downsample the data if needed
                if self.downsample != 1:
                    df = df.iloc[::self.downsample].reset_index(drop=True)
                    if debug:
                        print(f"After downsampling by factor {self.downsample}, rows: {len(df)}")
                
                df = df[df['Sleep_Stage'] != 'P'] # remove data before PSG start
                for col in ['ACC_X', 'ACC_Y', 'ACC_Z','BVP', 'TEMP', 'TIMESTAMP']:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
                ACC = np.sqrt(df['ACC_X']**2 + df['ACC_Y']**2 + df['ACC_Z']**2) # assuming its unlikely each acc channel really carries important information
                df_X = df[['TIMESTAMP', 'BVP', 'TEMP']].copy()
                df_X['ACC'] = ACC
                # Normalize the features (z-score normalization per subject)
                TEMP_norm = (df_X['TEMP'] - df_X['TEMP'].mean()) / df_X['TEMP'].std()
                df_X['TEMP'] = TEMP_norm
                BVP_norm = (df_X['BVP'] - df_X['BVP'].mean()) / df_X['BVP'].std()
                df_X['BVP'] = BVP_norm
                df['Sleep_Stage'] = df['Sleep_Stage'].astype(str).str.strip()
                df_Y = df['Sleep_Stage'].map(SLEEP_STAGE_MAPPING)
                
                # Pad/truncate the data to the downsampled max_length
                if len(df_X) > max_length:
                    if debug:
                        print(f"Truncating data for {SID} from {len(df_X)} to {max_length} samples.")
                    df_X = df_X.iloc[:max_length]
                    df_Y = df_Y.iloc[:max_length]
                else:
                    padding_length = max_length - len(df_X)
                    padding = pd.DataFrame(np.nan, index=np.arange(padding_length), columns=df_X.columns)
                    df_X = pd.concat([df_X, padding], ignore_index=True)
                    df_Y = pd.concat([df_Y, pd.Series([-1] * padding_length)], ignore_index=True)
                self.subjects[subjectNo] = {
                    'data': df_X.values.astype(np.float32),  # shape: [T, C]
                    'labels': df_Y.to_numpy(),                 # shape: [T]
                    'SID': SID
                }
                if debug:
                    print(f"Data shape for {SID}: {df_X.shape}, Labels shape: {df_Y.shape}")
            else:
                warning(f"File {file_path} does not exist. Skipping subject {SID}.")
    def __len__(self):
        return len(self.subjects)

    def __getitem__(self, idx):
        subject = self.subjects[idx]
        data = torch.tensor(subject['data'], dtype=torch.float32)
        labels = torch.tensor(subject['labels'], dtype=torch.long)

        data = forward_fill(data) # fill NaNs with previous values
        labels = forward_fill(labels) # fill NaNs with previous values
        return data, labels



In [53]:
# Sleep stage mapping as before
SLEEP_STAGE_MAPPING = {
    "W": 0,    # Wake
    "N1": 1,   # non-REM stage 1
    "N2": 2,   # non-REM stage 2
    "N3": 3,   # non-REM stage 3
    "R": 4,    # REM
    "Missing": -1  # Missing label
}

def forward_fill(x):
    """
    Performs forward fill on a tensor.
    If x is 1D (shape [T]), it is temporarily unsqueezed to [T, 1].
    Assumes the first value is valid, or fills it with zero if needed.
    """
    single_channel = False
    if x.dim() == 1:
        x = x.unsqueeze(1)
        single_channel = True

    T, C = x.shape
    for c in range(C):
        if torch.isnan(x[0, c]):
            x[0, c] = 0.0
        for t in range(1, T):
            if torch.isnan(x[t, c]):
                x[t, c] = x[t - 1, c]
    if single_channel:
        x = x.squeeze(1)
    return x

numeric_columns = [
    'TIMESTAMP', 'BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP',
    'EDA', 'HR', 'IBI'
]
converters = {col: safe_float for col in numeric_columns}

class SleepChunkDataset(Dataset):
    def __init__(self, subjects_list, data_dir, chunk_duration=600, chunk_stride=300, downsample_freq=64, debug=False):
        """
        Args:
            subjects_list (list): List of subject IDs, e.g. ["SID1", "SID2", ...].
            data_dir (str): Directory where files like "SID_whole_df.csv" are stored.
            chunk_duration (int): Chunk length in seconds (default 600 s for 10 minutes).
            chunk_stride (int): Time in seconds to step forward between chunks (default 300 s, for 50% overlap).
            downsample_freq (int): Desired sampling frequency after downsampling (original data are at 64 Hz).
            debug (bool): If True, print status messages.
        """
        self.chunks = []  # List to store each generated chunk (with its corresponding data, labels, and SID)
        # Compute downsample factor (original sampling rate is 64 Hz)
        self.downsample = int(64 // downsample_freq)
        # Effective sampling rate after downsampling becomes downsample_freq Hz.
        self.chunk_length = int(chunk_duration * downsample_freq)
        self.stride = int(chunk_stride * downsample_freq)

        for SID in subjects_list:
            file_path = os.path.join(data_dir, f"{SID}_whole_df.csv")
            if os.path.exists(file_path):
                df = pd.read_csv(file_path, dtype={'Sleep_Stage': 'category'}, converters=converters, low_memory=True)
                if debug:
                    print(f"Loaded data for subject {SID}")
                
                # Downsample: every self.downsample-th row
                if self.downsample != 1:
                    df = df.iloc[::self.downsample].reset_index(drop=True)
                    if debug:
                        print(f"After downsampling (factor {self.downsample}), rows: {len(df)}")
                
                # Remove rows with "Preparation" phase if labeled 'P'
                df = df[df['Sleep_Stage'] != 'P']

                # Ensure numeric conversion for required columns
                for col in ['ACC_X', 'ACC_Y', 'ACC_Z', 'BVP', 'TEMP', 'TIMESTAMP']:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
                
                # Compute accelerometer magnitude from three axes
                ACC = np.sqrt(df['ACC_X']**2 + df['ACC_Y']**2 + df['ACC_Z']**2)
                
                # Prepare the features: TIMESTAMP, BVP, TEMP and the computed ACC
                df_X = df[['TIMESTAMP', 'BVP', 'TEMP']].copy()
                df_X['ACC'] = ACC
                # Normalize the features (z-score normalization per subject)
                TEMP_norm = (df_X['TEMP'] - df_X['TEMP'].mean()) / df_X['TEMP'].std()
                df_X['TEMP'] = TEMP_norm
                BVP_norm = (df_X['BVP'] - df_X['BVP'].mean()) / df_X['BVP'].std()
                df_X['BVP'] = BVP_norm
                
                # Process sleep stage labels: trim whitespace and map to integer
                df['Sleep_Stage'] = df['Sleep_Stage'].astype(str).str.strip()
                df_Y = df['Sleep_Stage'].map(SLEEP_STAGE_MAPPING)
                
                # Convert features and labels to numpy arrays
                data_arr = df_X.values.astype(np.float32)  # shape: [T, C]
                labels_arr = df_Y.to_numpy()                # shape: [T]
                T = data_arr.shape[0]

                # If the record is too short (less than one chunk), pad it with NaNs (-1 for labels)
                if T < self.chunk_length:
                    pad_size = self.chunk_length - T
                    padding_data = np.full((pad_size, data_arr.shape[1]), np.nan, dtype=np.float32)
                    data_arr = np.concatenate([data_arr, padding_data], axis=0)
                    padding_labels = np.full((pad_size,), -1)
                    labels_arr = np.concatenate([labels_arr, padding_labels], axis=0)
                    T = self.chunk_length  # update length

                # Slide a window over the data with the defined stride to create overlapping chunks
                for start in range(0, T - self.chunk_length + 1, self.stride):
                    end = start + self.chunk_length
                    chunk_data = data_arr[start:end, :]
                    chunk_labels = labels_arr[start:end]
                    self.chunks.append({
                        'data': chunk_data,
                        'labels': chunk_labels,
                        'SID': SID
                    })
                if debug:
                    num_chunks = (T - self.chunk_length) // self.stride + 1
                    print(f"Subject {SID}: {T} samples processed, generated {num_chunks} chunks")
            else:
                print(f"File {file_path} does not exist. Skipping subject {SID}")

    def __len__(self):
        return len(self.chunks)

    def __getitem__(self, idx):
        chunk = self.chunks[idx]
        data = torch.tensor(chunk['data'], dtype=torch.float32)
        labels = torch.tensor(chunk['labels'], dtype=torch.long)
        # Use forward_fill to replace any NaNs with previous values.
        data = forward_fill(data)
        labels = forward_fill(labels)
        return data, labels


### Construct train, val, and test datasets and dataloaders

In [ ]:
'''
train_dataset_windowed = SleepDataset(subjects_list=subjects_train,
                                 data_dir=datadir_64Hz,
                                 window_size_ms= 20000, # 20 seconds of data
                                 stride_ms=5000,        # 5 seconds overlap
                                 downsample_freq=8, # downsample to 16Hz
                                 debug=False)
val_dataset_windowed = SleepDataset(subjects_list=subjects_val,
                                 data_dir=datadir_64Hz,
                                 window_size_ms= 20000, # 20 seconds of data
                                 stride_ms=5000,        # 5 seconds overlap
                                 downsample_freq=8, # downsample to 16Hz
                                 debug=False)
test_dataset_windowed = SleepDataset(subjects_list=subjects_test,
                                 data_dir=datadir_64Hz,
                                 window_size_ms= 20000, # 20 seconds of data
                                 stride_ms=5000,        # 5 second stride
                                 downsample_freq=8, # downsample to 16Hz
                                 debug=False)
'''

train_dataset = SleepDataset(subjects_list=subjects_train,
                                 data_dir=datadir_64Hz,
                                 max_length=max_length,
                                 downsample_freq=8, # downsample to 8Hz
                                 debug=False)

val_dataset = SleepDataset(subjects_list=subjects_val,
                                 data_dir=datadir_64Hz,
                                 max_length=max_length,
                                 downsample_freq=8, # downsample to 8Hz
                                 debug=False)
test_dataset = SleepDataset(subjects_list=subjects_test,
                                 data_dir=datadir_64Hz,
                                 max_length=max_length,
                                 downsample_freq=8, # downsample to 8Hz
                                 debug=False)

print(f"Total samples in train dataset: {len(train_dataset)}")
print(f"Total samples in val dataset: {len(val_dataset)}")
print(f"Total samples in test dataset: {len(test_dataset)}")


Total samples in train dataset: 80
Total samples in val dataset: 10
Total samples in test dataset: 10


In [54]:
train_chunk_dataset = SleepChunkDataset(subjects_list=subjects_train,
                                 data_dir=datadir_64Hz,
                                 chunk_duration=600,  # 10 minutes
                                 chunk_stride=300,    # 5 minutes
                                 downsample_freq=0.2,   # downsample to 0.2 Hz
                                 debug=False)
val_chunk_dataset = SleepChunkDataset(subjects_list=subjects_val,
                                 data_dir=datadir_64Hz,
                                 chunk_duration=600,  # 10 minutes
                                 chunk_stride=300,    # 5 minutes
                                 downsample_freq=0.2,   # downsample to 0.2 Hz
                                 debug=False)
test_chunk_dataset = SleepChunkDataset(subjects_list=subjects_test,
                                 data_dir=datadir_64Hz,
                                 chunk_duration=600,  # 10 minutes
                                 chunk_stride=300,    # 5 minutes
                                 downsample_freq=0.2,   # downsample to 0.2 Hz
                                 debug=False)
print(f"Total samples in train chunk dataset: {len(train_chunk_dataset)}")
print(f"Total samples in val chunk dataset: {len(val_chunk_dataset)}")
print(f"Total samples in test chunk dataset: {len(test_chunk_dataset)}")

Total samples in train chunk dataset: 6547
Total samples in val chunk dataset: 821
Total samples in test chunk dataset: 845


## CNN downsampling approach

In [ ]:
# sequence length x input channels -> CNN -> shortened sequence length x num hidden channels -> LSTM -> shortened sequence length x num sleep stages

class FeatureExtractorCNN(nn.Module):
    def __init__(self, in_channels=4, cnn_output_channels=128):
        super(FeatureExtractorCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, 32, kernel_size=5, stride=2, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=2, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.conv3 = nn.Conv1d(64, 64, kernel_size=5, stride=2, padding=1)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.conv4 = nn.Conv1d(64, cnn_output_channels, kernel_size=3, stride=1, padding=1)
        self.bn = nn.BatchNorm1d(cnn_output_channels)
        self.relu = nn.ReLU()
        # Global average pooling to collapse 
    
    def forward(self, x):
        assert not torch.isnan(x).any(), "NaN detected in CNN input"
        # Expect x of shape (batch, epoch_samples, channels)
        x = x.permute(0, 2, 1)  # Rearrange to (batch, channels, epoch_samples)
        #print(f"Input shape after permutation: {x.shape}")
        x = self.relu(self.conv1(x))
        x = self.pool1(x)
        #print(f"Shape after conv1 and pool1: {x.shape}")
        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        #print(f"Shape after conv2 and pool2: {x.shape}")
        x = self.relu(self.conv3(x))
        x = self.pool3(x)
        #print(f"Shape after conv3 and pool3: {x.shape}")
        x = self.relu(self.conv4(x))
        #print(f"Shape after conv4: {x.shape}")
        x = self.bn(x)

        return x

class SleepStageLSTM(nn.Module):
    def __init__(self, cnn_output_channels=128, hidden_size=64, num_layers=2, num_sleep_stages=5):
        super(SleepStageLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=cnn_output_channels,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=False)
        self.fc = nn.Linear(hidden_size, num_sleep_stages)
    
    def forward(self, x):
        assert not torch.isnan(x).any(), "NaN detected in LSTM input"
        # x is of shape (batch, cnn_output_channels, samples)
        # LSTM expects input shape of (samples, batch, cnn_output_channels)
        x = x.permute(2, 0, 1) # (batch, cnn_output_channels, samples) -> (samples, batch, cnn_output_channels)
        #print(f"Input shape after permutation: {x.shape}")
        lstm_out, _ = self.lstm(x)
        #print(f"Shape after LSTM: {lstm_out.shape}")
        # Option 1: produce a prediction for every epoch (each time step)
        out = self.fc(lstm_out)   # shape: (batch, num_epochs, num_sleep_stages)
        #print(f"Shape after fully connected layer: {out.shape}")
        
        # Option 2: if you want a prediction only for the current epoch,
        # you may take the output of the last time step:
        #predictions = self.fc(lstm_out[:, -1, :])  # shape: (batch, num_sleep_stages)
        return out

class OnlineSleepStagingModel(pl.LightningModule):
    def __init__(self, in_channels, cnn_output_channels, lstm_hidden_size, num_layers=2, num_sleep_stages=5, learning_rate=0.001):
        super(OnlineSleepStagingModel, self).__init__()
        self.save_hyperparameters()
        self.feature_extractor = FeatureExtractorCNN(in_channels=4, cnn_output_channels=cnn_output_channels)
        self.lstm_model = SleepStageLSTM(cnn_output_channels=cnn_output_channels, hidden_size=lstm_hidden_size, num_layers=num_layers, num_sleep_stages=num_sleep_stages)
        self.learning_rate = learning_rate
        self.criterion = nn.CrossEntropyLoss(ignore_index=-1)  # Ignore the "Missing" label (-1)
        self.num_sleep_stages = num_sleep_stages
        self.cnn_output_channels = cnn_output_channels

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.lstm_model(x) # (samples, batch_size, num_sleep_stages)
        assert x.shape[2] == self.num_sleep_stages, "LSTM output shape != num_sleep_stages"
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch  # y shape: [batch_size, T]
        y_hat = self(x)  # y_hat shape: [output_length, batch_size, num_sleep_stages]
        # Check for NaNs in the network output
        assert not torch.isnan(y).any(), "NaN detected in labels"
        assert not torch.isnan(y_hat).any(), "NaN detected in network output"
    
        # Permute to batch first
        y_hat = y_hat.permute(1, 0, 2)
        output_length = y_hat.shape[1]
        y_expanded = y.unsqueeze(1)
        # Downsample y to match y_hat
        y_resampled = torch.nn.functional.interpolate(
            y_expanded.float(),
            size = (output_length,),
            mode = 'nearest'
        )
        y_resampled = y_resampled.squeeze(1).long()

        # Flatten y_hat and y_resampled for loss calculation
        batch_size, output_length, num_sleep_stages = y_hat.shape
        y_hat_flat = y_hat.reshape(batch_size * output_length, num_sleep_stages)
        y_resampled_flat = y_resampled.reshape(batch_size * output_length)
        # Calculate loss
        loss = self.criterion(y_hat_flat, y_resampled_flat)
        # Check loss for finiteness
        assert torch.isfinite(loss), "Loss is not finite"
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch  # y shape: [batch_size, T]
        y_hat = self(x)  # y_hat shape: [output_length, batch_size, num_sleep_stages]
        # Permute to batch first
        y_hat = y_hat.permute(1, 0, 2)
        output_length = y_hat.shape[1]
        y_expanded = y.unsqueeze(1)
        # Downsample y to match y_hat
        y_resampled = torch.nn.functional.interpolate(
            y_expanded.float(),
            size = (output_length,),
            mode = 'nearest'
        )
        y_resampled = y_resampled.squeeze(1).long()

        # Flatten y_hat and y_resampled for loss calculation
        batch_size, output_length, num_sleep_stages = y_hat.shape
        y_hat_flat = y_hat.reshape(batch_size * output_length, num_sleep_stages)
        y_resampled_flat = y_resampled.reshape(batch_size * output_length)
        predictions = torch.argmax(y_hat_flat, dim=1)
        # Calculate Cohen's Kappa
        #kappa = MulticlassCohenKappa(num_classes=self.num_sleep_stages)
        #cohen_kappa_score = kappa(predictions, y_resampled_flat)
        #self.log("val_cohen_kappa", cohen_kappa_score, prog_bar=True)
        # Calculate loss
        loss = self.criterion(y_hat_flat, y_resampled_flat)
        self.log("val_loss", loss, prog_bar=True)
        return loss
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

class CNNClassifier(pl.LightningModule):
    def __init__(self, in_channels, cnn_output_channels, lstm_hidden_size, num_layers=2, num_sleep_stages=5, learning_rate=0.001):
        super(CNNClassifier, self).__init__()
        self.save_hyperparameters()
        self.feature_extractor = FeatureExtractorCNN(in_channels=4, cnn_output_channels=cnn_output_channels)
        self.lstm_model = SleepStageLSTM(cnn_output_channels=cnn_output_channels, hidden_size=lstm_hidden_size, num_layers=num_layers, num_sleep_stages=num_sleep_stages)
        self.learning_rate = learning_rate
        self.criterion = nn.CrossEntropyLoss(ignore_index=-1)  # Ignore the "Missing" label (-1)
        self.num_sleep_stages = num_sleep_stages
        self.cnn_output_channels = cnn_output_channels

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.lstm_model(x) # (samples, batch_size, num_sleep_stages)
        assert x.shape[2] == self.num_sleep_stages, "LSTM output shape != num_sleep_stages"
        return x



In [35]:
# demo of model elements
temp_input = train_dataset[0][0]
print(f"Input shape: {temp_input.shape} (epoch_samples, channels)")
CNN_model = FeatureExtractorCNN(in_channels=4, cnn_output_channels=128)
cnn_output = CNN_model(temp_input.unsqueeze(0))
print(f"Output shape: {cnn_output.shape} (batch_size, cnn_output_channels, epoch_samples)")
LSTM_model = SleepStageLSTM(cnn_output_channels=128, hidden_size=64, num_layers=2, num_sleep_stages=5)
LSTM_output = LSTM_model(cnn_output)

# demo of combined model
combined_model = OnlineSleepStagingModel(in_channels=4, cnn_output_channels=128, lstm_hidden_size=64, num_layers=2, num_sleep_stages=5)
combined_output = combined_model(temp_input.unsqueeze(0))
print(f"Combined model output shape: {combined_output.shape} (samples, batch_size, num_sleep_stages)")
print(combined_output.shape[0])

temp_labels = train_dataset[0][1]
print(f"Labels shape: {temp_labels.shape} (epoch_samples)")


Input shape: torch.Size([2493810, 4]) (epoch_samples, channels)
Output shape: torch.Size([1, 128, 38965]) (batch_size, cnn_output_channels, epoch_samples)
Combined model output shape: torch.Size([38965, 1, 5]) (samples, batch_size, num_sleep_stages)
38965


KeyboardInterrupt: 

In [56]:
print(temp_input.shape[0] / combined_output.shape[0])

wandb.finish()

64.00128320287438


In [59]:
# Train the combined model
wandb_logger = WandbLogger(name="online_sleep_staging_model", project="sleep_stage_classification")
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/',
    filename='best-checkpoint',
    save_top_k=1,
    mode='min'
)
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=True,
    mode='min'
)
trainer = pl.Trainer(
    max_epochs=10,
    devices=1,
    accelerator='gpu',
    logger=wandb_logger,
    callbacks=[checkpoint_callback, early_stop_callback]
)
model = OnlineSleepStagingModel(in_channels=4, cnn_output_channels=16, lstm_hidden_size=64, num_layers=2, num_sleep_stages=5, learning_rate=1e-4)
train_loader = DataLoader(train_chunk_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_chunk_dataset, batch_size=8, shuffle=False)
trainer.fit(model, train_loader, val_loader)
wandb.finish()
# Load the best model
best_model_path = checkpoint_callback.best_model_path
best_model = OnlineSleepStagingModel.load_from_checkpoint(best_model_path)


/gpfs/data/oermannlab/users/slj9342/.conda/envs/dl4med_25/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /gpfs/data/oermannlab/users/slj9342/.conda/envs/dl4m ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


/gpfs/data/oermannlab/users/slj9342/.conda/envs/dl4med_25/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /gpfs/data/oermannlab/users/slj9342/dl4med_25/dl4med_final_project/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                | Params | Mode 
------------------------------------------------------------------
0 | feature_extractor | FeatureExtractorCNN | 34.6 K | train
1 | lstm_model        | SleepStageLSTM      | 54.6 K | train
2 | criterion         | CrossEntropyLoss    | 0      | train
------------------------------------------------------------------
89.2 K    Trainable params
0         Non-trainable params
89.2 K    Total params
0.357     Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 1.336. Signaling Trainer to stop.


epoch,▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▅▅▅▅▆▆▆▆▇▇▇▇▇▇███████
train_loss_epoch,█▂▁▁▁▁▁
train_loss_step,▅▄▃▃▄▂▂▃█▄▄▄▂▄▅▄▂▃▂▃▄▄▃▅▃▃▃▆▃▄▄▄▁▂▃▃▂▃▃▂
trainer/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
val_loss,▇▃▂▁▂▂█
epoch,6
train_loss_epoch,1.24735
train_loss_step,1.0131
trainer/global_step,5732
val_loss,1.36093


In [ ]:
import optuna

def objective(trial):
    # Sample hyperparameters
    cnn_output_channels = trial.suggest_categorical("cnn_output_channels", [8, 16, 32, 64])
    lstm_hidden_size = trial.suggest_categorical("lstm_hidden_size", [32, 64, 128])
    num_layers = 2
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)

    wandb_logger = WandbLogger(name=f"CNN{cnn_output_channels}_hs{lstm_hidden_size}_lr{learning_rate}", project="optuna_sleep_stage_classification")
    # DataLoaders (resample based on batch size)
    train_loader = DataLoader(train_chunk_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_chunk_dataset, batch_size=16, shuffle=False)

    # Model
    model = OnlineSleepStagingModel(
        in_channels=4,
        cnn_output_channels=cnn_output_channels,
        lstm_hidden_size=lstm_hidden_size,
        num_layers=num_layers,
        num_sleep_stages=5,
        learning_rate=learning_rate,
    )

    # Trainer with pruning callback
    trainer = pl.Trainer(
        max_epochs=10,
        devices=1,
        accelerator="gpu",
        logger=wandb_logger,
        enable_checkpointing=False,
        callbacks=[
            EarlyStopping(monitor="val_loss", patience=3, mode="min")
        ]
    )

    trainer.fit(model, train_loader, val_loader)
    return trainer.callback_metrics["val_loss"].item()

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

[I 2025-04-14 16:17:15,889] A new study created in memory with name: no-name-7ca2f822-327a-4fcb-857f-0c56425d76e2
/tmp/ipykernel_2298590/1061032611.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
/gpfs/data/oermannlab/users/slj9342/.conda/envs/dl4med_25/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /gpfs/data/oermannlab/users/slj9342/.conda/envs/dl4m ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                | Params | Mode 
------------------------------------------------------------------
0 | feature_extractor | FeatureExtractorCNN | 37.8 K | train
1 | lstm_model        | SleepStageLSTM      | 215 K  | train
2 | criterion         | CrossEntropyLoss    | 0      | train
------------------------------------------------------------------
253 K     Trainable params
0         Non-trainable params
253 K     Total params
1.014     Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# project overview
# construct dataset class
# - train CNN -> LSTM sleep staging model with causal conv and unidirectional LSTM
# - train CNN -> LSTM sleep staging model with non-causal conv and bidirectional LSTM
# - compare performance at 64Hz
# - compare performance and computational cost at lower sampling rates

# unstructured notes
# - likely best to combine ACC columns into a single variable. Can't imagine they offer much additional information
# - CNN model should use 2D CNNs to extract features across both channels and short-term time windows
# - will need to pad data on both sides to ensure input and output sequences for the CNN are the same length


# CNN model - input: seq_len x num_channels -> output: seq_len x hidden_size


# LSTM model - input: seq_len x hidden_size -> output: seq_len x num_classes




In [ ]:
# separate data into train val and test by subject
participant_info_df = pd.read_csv('/gpfs/data/oermannlab/users/slj9342/dl4med_25/data/physionet.org/files/dreamt/2.0.0/participant_info.csv')
subject_ids = participant_info_df['subject_id'].unique()
train_subjects = subject_ids[:int(len(subject_ids)*0.8)]
val_subjects = subject_ids[int(len(subject_ids)*0.8):int(len(subject_ids)*0.9)]
test_subjects = subject_ids[int(len(subject_ids)*0.9):]

In [ ]:
print(demo_df.columns)
print(demo_df['Sleep_Stage'].unique())
print(demo_df['TIMESTAMP'].head())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
sleep_stage_labels = le.fit_transform(demo_df['Sleep_Stage'])
plt.plot(sleep_stage_labels)
plt.title('Sleep Stage Labels')
plt.xlabel('Time')
plt.ylabel('Sleep Stage')
plt.show()

In [ ]:
import pytorch_lightning as pl
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

cols_X = ['TIMESTAMP', 'BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP']
cols_Y = ['Sleep_Stage']

class DreamtDataset(Dataset):
    def __init__(self, data_dir, subject_ids, cols_X, cols_Y):
        self.data_dir = data_dir
        self.subject_ids = subject_ids
        self.cols_X = cols_X
        self.cols_Y = cols_Y
        self.data = []
        for subject_id in subject_ids:
            df = pd.read_csv(data_dir + f'S{subject_id:03d}_whole_df.csv')
            df = df[cols_X + cols_Y]
            df['subject_id'] = subject_id
            self.data.append(df)
        self.data = pd.concat(self.data, ignore_index=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        X = torch.tensor(row[self.cols_X].values, dtype=torch.float32)
        y = torch.tensor(row[self.cols_Y].values, dtype=torch.long)
        return X, y

# CNN model with 4 2D conv layers, relu activation, dropout of 0.2, and 2 fully connected layers, first one has size 4096, second with 1500, both fc layers have dropout of 0.5
# last fc layer has output size of 5, the number of channels to feed into the LSTM
import torch
import torch.nn as nn
import torch.nn.functional as F

# Input shape: (batch_size, 1, num_channels, time_window)
# Example: 6 EEG channels, 128-sample time windows → (batch, 1, 6, 128)

import torch
import torch.nn as nn
import torch.nn.functional as F

def sliding_windows(x, window_size, stride=1):
    # x: (batch, channels, time)
    batch, channels, total_len = x.shape

    # Pad on both sides: same number of windows as original time length
    pad = (window_size - 1) // 2
    x_padded = F.pad(x, pad=(pad, pad), mode='replicate')

    # Now generate sliding windows
    windows = x_padded.unfold(dimension=2, size=window_size, step=stride)  # (batch, channels, time, window_size)
    windows = windows.permute(0, 2, 1, 3)  # (batch, time, channels, window_size)
    return windows  # shape: (batch, time, channels, window_size)



class DreamtCNN(nn.Module):
    def __init__(self, input_shape, num_chans_out):
        super(DreamtCNN, self).__init__()
        c, t = input_shape  # c = number of channels (e.g. 6), t = time_window size (e.g. 128)

        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 5), stride=1, padding=(1, 2))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 5), stride=1, padding=(1, 2))
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(3, 5), stride=1, padding=(1, 2))
        self.conv4 = nn.Conv2d(128, 256, kernel_size=(3, 5), stride=1, padding=(1, 2))

        self.conv_dropout = nn.Dropout(0.2)
        self.ff_dropout = nn.Dropout(0.5)

        # Output dims after conv layers will be (batch, 256, c, t)
        self.fc1 = nn.Linear(256 * c * t, 4096)
        self.fc2 = nn.Linear(4096, num_chans_out)

    def forward(self, x):
        # x shape: (batch, 1, channels, time_window)
        x = F.relu(self.conv1(x))
        x = self.conv_dropout(x)
        x = F.relu(self.conv2(x))
        x = self.conv_dropout(x)
        x = F.relu(self.conv3(x))
        x = self.conv_dropout(x)
        x = F.relu(self.conv4(x))
        x = self.conv_dropout(x)

        x = x.view(x.size(0), -1)  # flatten
        x = F.relu(self.fc1(x))
        x = self.ff_dropout(x)
        x = self.fc2(x)  # output shape: (batch_size, num_chans_out)

        return x


# Define test parameters
batch_size = 4
num_channels = 6
time_window = 128
seq_len = 1000  # e.g., length of the sequence to be fed into the LSTM
num_chans_out = 5  # e.g., number of channels to feed into the LSTM

# create dummy imput 
x = torch.randn(batch_size, 1, num_channels, time_window)  # (batch_size, 1, num_channels, time_window)

# window input
x = x.squeeze(1)  # (batch_size, num_channels, time_window)
x = sliding_windows(x, window_size=time_window, stride=1)

print(f"Shape after sliding windows: {x.shape}")

# Instantiate the model
model = DreamtCNN(input_shape=(num_channels, time_window), num_chans_out=num_chans_out)

# Forward pass
# x shape: (batch, time, channels, window)
x = x.unsqueeze(2)  # -> (batch, time, 1, channels, window)

# Reshape for CNN: merge batch and time into one dim
b, t, one, c, w = x.shape
x = x.view(b * t, 1, c, w)  # (b * t, 1, channels, window)

# Feed into CNN
out = model(x)  # shape: (b * t, output_dim)

# Restore sequence shape
out = out.view(b, t, -1)  # (batch, time, output_dim)


# Print shape
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")


